In [ ]:
import sys
import numpy as np
from itertools import permutations
import regex as re
import os

from sklearn.tree import DecisionTreeClassifier, plot_tree
import pandas as pd

# Hamiltonian of few layer graphene

The reasons to why we build the Hamiltonian the way we do, can be found in my MSc Thesis. This section is ment to introduce the functions that create the momentum-space Hamiltonian. The calculations were done on a remote computer using the PDOS_calc.py file.

In [ ]:
def hk(D,t0,t1,t3,t4,N,in_list,k):
    '''
    This function creates a Hamiltonian for each k point sampled in the Brillouin zone.
    Parameters: 
        -D: on-site energy
        -t0,t1,t3,t4: hopping energies (notation weirdness comes from literature, where t2 is interaction of non-adjecent layers)
        -N: number of layers
        -in_list: list of faults, eg.:['1b','3b']
        -k: k-points with shape k[0] = x-coordinate k[1] = y-coordinate (output of grid_maker function)
    '''
    kx=k[0]
    ky=k[1]
    F = 1 + np.exp(1j/2*(np.sqrt(3)*kx+ky)) + np.exp(1j/2*(np.sqrt(3)*kx-ky)) #First phase factor
    F_A1B2 = np.exp(1j*ky*3) + np.exp(1j/2*(np.sqrt(3)*kx+ky)) + np.exp(1j/2*(np.sqrt(3)*kx-ky)) #Second phase factor
    
    #All sub-matrices are created first
    h0_ABC = np.array([[0*np.ones(np.shape(F)),t0*F], #Intra-layer  
                       [np.conj(t0*F),0*np.ones(np.shape(F))]]).transpose((2,0,1))
    
    D_mat = np.array([[0*np.ones(np.shape(F)),0*np.ones(np.shape(F))], #On-site
                      [0*np.ones(np.shape(F)),D*np.ones(np.shape(F))]]).transpose((2,0,1))
    
    D_mat_b = np.array([[D*np.ones(np.shape(F)),0*np.ones(np.shape(F))], #On-site in case of stacking fault
                      [0*np.ones(np.shape(F)),0*np.ones(np.shape(F))]]).transpose((2,0,1))
    
    h1 = np.array([[t4*F,t3*F_A1B2], #Inter-layer
                   [t1*np.ones(np.shape(F)),t4*F]]).transpose((2,0,1))
    
    h1_b = np.conj(np.array([[t4*F,t1*np.ones(np.shape(F))], #Inter-layer in case of stacking fault
                             [t3*F_A1B2,t4*F]])).transpose((2,0,1))
    
    
    H0k = np.zeros((len(F),2*N,2*N),dtype=complex) #The whole matrix, first created with zeros in the right shape
    
    onsite_mat = np.kron(np.eye(N),h0_ABC) #making it the right size
    
    #This part is handling stacking faults
    on_site_list = np.ones(N)
    on_site_list[-1] = 0
    on_site_list_b     = np.ones(N)
    on_site_list_b[0] = 0
    hoppingok_list = np.ones(N-1) 
    flag_T = False
    print(in_list)
    if (in_list == ['']): #This is just an ugly bug-fix
        in_list = []
    for string in in_list:
        helyzet = (int(re.search(r'\d+', string).group())) #Interpretation of input_list 
        hoppingok_list[helyzet-1]   -= 1
        on_site_list[helyzet-1]   -= 1
        on_site_list[helyzet]     += 1
        on_site_list_b[helyzet-1] += 1
        on_site_list_b[helyzet]   -= 1
        #This is to create vectors which have ones in the correct places, so we can make a kronecher product 
    
    hopping_mat = np.kron(np.diag(hoppingok_list,k=1),h1) + np.kron(np.diag(1-hoppingok_list,k=1),h1_b)
    hopping_mat += np.conj(np.transpose(hopping_mat,[0,2,1]))
    onsite_mat += np.kron(np.diag(on_site_list),D_mat) + np.kron(np.diag(on_site_list_b),D_mat_b)
    #To prove that this is the correct way of building the matrices is left as an excercise for the reader 
    
    H0k += onsite_mat
    H0k += hopping_mat #Adding up the matrix
    return H0k

def grid_maker(vec1,vec2,num,misplace=[0,0]):
    '''
    This function creates a grid of the paralelogramm created of two vectors
    '''
    points = []
    for i in range(num):
        for j in range(num):
            points.append([misplace[0] + i*vec1[0]/num + j*vec2[0]/num,misplace[1] + i*vec1[1]/num + j*vec2[1]/num])
    return np.array(points).reshape(num*num,2)

# Computing all possible stacking configurations up to N layers

We rule out AA stacking since it is energetically less favourable. After i number of layers the i+1th can be hence put either to the left or the right of the previous one. Let us denote right +1 and left -1, which enables us to encode each configuration with a string of +1s and -1s.
$$\mathrm{ABA} \Rightarrow -1+1 = +1-1$$
$$\mathrm{ABC} \Rightarrow +1+1 = -1-1$$
Configurations that are topologically not different will result in the same PDOS. Finding all topologically different configurations is an important task before the calculations since it decreases computation time significantly.
Topologically different means in this case that the strings can not be transformed into each other with flipping the signs or reversing the order or both. (In fact, these operations are equivalent with mirroring and rotating the geometry itself.) 

In [ ]:
def create_combination_list(N):
    '''
    Create a list containing all topologically different combinations of each layer up to N in the format required for in_list.
    For example the 5th element of the output list has all possible configurations for N=5 layers.
    '''
    ossz_kombok_unzipped = [] #store combination lists
    for N in range(0,11):
        faults = np.array([f'{i}b' for i in range(1,N)])
        perms = []
        kombok = []
        #These cycles check the conditions mentioned above
        for i in range(int(np.ceil(N/2))):
            for perm in sorted(set(permutations(np.concatenate((-1*np.ones(i),np.ones(N-1-i)))))):
                if list(perm)[::-1] not in perms and list(-1*np.array(perm)) not in perms:
                    perms.append(list(perm))
                    kombok.append((faults[np.array(perm) == -1]).tolist())
        ossz_kombok_unzipped.append(kombok)    
    return ossz_kombok_unzipped

# Computing projected density of states (PDOS)

The PDOS is the DOS weighted with the absolute value squared of the eigenvectors. In the numerical calculations the Dirac delta in the definition of DOS can be substituted with a narrow Gaussian:
$$\mathrm{PDOS}_n = N \sum_i^{\mathrm{bands}}\sum_{j=1}^N |\Psi_n|^2\frac{1}{\alpha\sqrt{\pi}}e^{-\frac{(E-E_i)^2}{\alpha^2}}$$


In [ ]:
def PDOS(E,Energies,w,alpha,norm):
    '''
    Function to calculate the PDOS in the energy range E from the eigenvalues and eigenvectors of the Hamiltonian created.
    Parameters:
        -E: energy range
        -Energies: eigenvalues
        -w: eigenvectors
        -alpha: smoothing factor
        -norm: could be calculated, but left as 1 in the work done for the thesis
    '''
    Energies = Energies.flatten()
    w = np.transpose(w, axes=[0,2,1]) #this is to have the vectors in the correct shape. If not gotten from np.eigh it might be buggy
    w = abs(w.reshape(w.shape[0]*w.shape[1], w.shape[2]))**2
    D = norm/alpha * w.T @ np.exp(-((Energies[:,None] - E)**2)/(2*alpha**2))
    return D

# Evaluation with tree model

This part showcases a very rudimental application of the decision tree algorithm offered by sklearn. After the PDOS spectra are obtained it proved to be very complicated to check 200+ stacking configurations by hand and identify the substructures in them, thus a decisiontree was used.

In [ ]:
'''
This part is very specific to my dataset. If it were to be recreated than these steps could be omitted
'''
directories_old = [name for name in os.listdir("sergej_gamma_1001_E_newcombs//sergej_gamma_1001_E//") if os.path.isdir(f'sergej_gamma_1001_E_newcombs//sergej_gamma_1001_E//{name}')]
pdos_list_old = [genfromtxt(f'sergej_gamma_1001_E_newcombs//sergej_gamma_1001_E//{s}//pdos_normed.dat',delimiter=';') for s in directories_old]
directories_new = [name for name in os.listdir("getdicts//") if os.path.isdir(f'getdicts//{name}')]
directories = [directory for directory in directories_old if directory in directories_new]
pdos_list = [pdos for i,pdos in enumerate(pdos_list_old) if directories_old[i] in directories_new]
'''
This is the poin where the reading in of the dataset is done.
'''
E = linspace(-1,1,1001) #the energy range of the calculations
faultok_list = [s.split('[', 1)[1].split(']')[0].split(', ') for s in directories] #which faults were used
N_list = [int(s.split('_')[0]) for s in directories] 

'''
We only use the sum of the top and bottom layers' sublattice points
'''
pdos_top_sum_list = [pdos[-1] + pdos[-2] for pdos in pdos_list]
pdos_bot_sum_list = [pdos[0] + pdos[1] for pdos in pdos_list]

'''
Only the middle of the spectrum is important
'''
pdos_top_sum_cut_list = [pdos_top_sum[len(pdos_top_sum)//4:3*len(pdos_top_sum)//4] for pdos_top_sum in pdos_top_sum_list]
pdos_bot_sum_cut_list = [pdos_bot_sum[len(pdos_bot_sum)//4:3*len(pdos_bot_sum)//4] for pdos_bot_sum in pdos_bot_sum_list]

'''
Preparing the dataset to be used
'''
pdos_top_sum_df = pd.DataFrame(data=pdos_top_sum_cut_list,columns=E[len(E)//4:3*len(E)//4])
pdos_bot_sum_df = pd.DataFrame(data=pdos_bot_sum_cut_list,columns=E[len(E)//4:3*len(E)//4]) #pandas dataframes can be easily handled by sklearn
pdos_top_sum_df_train = pd.concat((pdos_top_sum_df[136:],pdos_bot_sum_df[136:]),ignore_index=True) #train dataset, omitting 10 layers
pdos_top_bot_sum_df_train = pdos_top_sum_df_train.round(5).drop_duplicates() #dropping duplicates, this helps the learning algorithm

y_train_top = ['TOP_' + directory for directory in directories[136:]]
y_train_bot = ['BOT_' + directory for directory in directories[136:]]
y_train = y_train_top + y_train_bot 

y_test_top = ['TOP_' + directory for directory in directories[:136]]
y_test_bot = ['BOT_' + directory for directory in directories[:136]]
y_test = y_test_top + y_test_bot #Giving proper class names

y_train = array(y_train)[pdos_top_bot_sum_df_train.index] #this solves some issues I guess

'''
The training
'''
clf = DecisionTreeClassifier(random_state=69420).fit(pdos_top_bot_sum_df_train,y_train) #the built-in decision tree

'''
Prediction dataset
'''
pdos_test = pd.concat((pdos_top_sum_df[:136],pdos_bot_sum_df[:136]),ignore_index=True).round(5)
directories_pred_abs = clf.predict(pdos_test)

'''
The evaluation was done by hand.
'''